In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avjanata-hackhealth-care-analytics2/train_data_dict.csv
/kaggle/input/avjanata-hackhealth-care-analytics2/sample_submission_lfbv3c3.csv
/kaggle/input/avjanata-hackhealth-care-analytics2/test.csv
/kaggle/input/avjanata-hackhealth-care-analytics2/train.csv


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold,KFold,GroupKFold
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings("ignore")
import time

In [3]:
train=pd.read_csv('/kaggle/input/avjanata-hackhealth-care-analytics2/train.csv')
test=pd.read_csv('/kaggle/input/avjanata-hackhealth-care-analytics2/test.csv')
sub=pd.read_csv('/kaggle/input/avjanata-hackhealth-care-analytics2/sample_submission_lfbv3c3.csv')
des=pd.read_csv('/kaggle/input/avjanata-hackhealth-care-analytics2/train_data_dict.csv')

In [4]:
train.head(2)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50


In [5]:
test.head(2)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,318439,21,c,3,Z,3,gynecology,S,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0
1,318440,29,a,4,X,2,gynecology,S,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0


In [6]:
len(train),len(test)

(318438, 137057)

In [7]:
train.isnull().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    4532
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [8]:
test.isnull().sum()

case_id                                 0
Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                              35
patientid                               0
City_Code_Patient                    2157
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
dtype: int64

### Concating

In [9]:
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])

### Interaction Feature

In [10]:
# df['patientid']=df['patientid'].astype(str)
# df['Hospital_code']=df['Hospital_code'].astype(str)

In [11]:
# df['hc_pi']=df['Type of Admission']+'_'+df['Severity of Illness']

In [12]:
# df['patientid']=df['patientid'].astype(int)
# df['Hospital_code']=df['Hospital_code'].astype(int)

### Label Encoding

In [13]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()

# for col in ['Hospital_type_code','Hospital_region_code', 'Available Extra Rooms in Hospital','Department', 'Ward_Type', 'Ward_Facility_Code','Type of Admission','patientid','Stay']:
#     df[col]=  df[col].astype('str')
#     df[col]= le.fit_transform(df[col])

### Mapping

In [14]:
hospital_type_map={
 'a': 0,
 'b': 1,
 'c': 2,
 'e': 3,
 'd': 4,
 'f': 5,
 'g': 6} 

hospital_region_map = {'X': 0, 'Y': 1, 'Z': 2}

dep_map={'gynecology': 0,
 'anesthesia': 1,
 'radiotherapy': 2,
 'TB & Chest disease': 3,
 'surgery': 4}

ward_type_map ={'R': 0, 'Q': 1, 'S': 2, 'P': 3, 'T': 4, 'U': 5}
ward_fac_map ={'F':0, 'E': 1, 'D':2, 'C': 3, 'B': 4, 'A': 5}
admiss_map = {'Trauma': 0, 'Emergency': 1, 'Urgent': 2}
stay_map = {'21-30': 2,
 '11-20': 1,
 '31-40': 3,
 '51-60': 5,
 '0-10': 0,
 '41-50': 4,
 '71-80': 7,
 'More than 100 Days': 10,
 '81-90': 8,
 '91-100': 9,
 '61-70': 6}

In [15]:
df['Hospital_type_code'] = df['Hospital_type_code'].map(hospital_type_map)
df['Hospital_region_code'] = df['Hospital_region_code'].map(hospital_region_map)
df['Department'] = df['Department'].map(dep_map)
df['Ward_Type'] = df['Ward_Type'].map(ward_type_map)
df['Ward_Facility_Code'] = df['Ward_Facility_Code'].map(ward_fac_map)
df['Type of Admission'] = df['Type of Admission'].map(admiss_map)
df['Stay'] = df['Stay'].map(stay_map)

In [16]:
df['Severity of Illness'] = df['Severity of Illness'].map({'Minor': 1,
                          'Moderate ': 2,
                          'Extreme': 3})

In [17]:
df['Age'] = df['Age'].map({'0-10': 5,
                          '11-20': 15,
                          '21-30': 25,
                          '31-40': 35,
                          '41-50': 45,
                          '51-60': 55,
                          '61-70': 65,
                          '71-80': 75,
                          '81-90': 85,
                          '91-100': 95
                         })

### Frequency Encoding

In [18]:
# fe_pol = (df.groupby('hc_pi').size()) / len(df)
# df['hc_pi'] = df['hc_pi'].apply(lambda x : fe_pol[x])

### Feature Engineering

In [19]:
df.head(2)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,train_or_test
0,1,8,2,3,2,3,2,0,0,2.0,31397,7.0,1,3.0,2,55,4911.0,0.0,train
1,2,2,2,5,2,2,2,2,0,2.0,31397,7.0,0,3.0,2,55,5954.0,4.0,train


In [20]:
df['mean_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('mean')
df['mean_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('mean')

In [21]:
df['sum_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('sum')
df['sum_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('sum')

In [22]:
df['max_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('max')
df['max_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('max')

In [23]:
df['min_Admission_Deposit_per_illness']=df.groupby(['Severity of Illness'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_admission']=df.groupby(['Type of Admission'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_bed']=df.groupby(['Bed Grade'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_department']=df.groupby(['Department'])['Admission_Deposit'].transform('min')
df['min_Admission_Deposit_per_hosp']=df.groupby(['Hospital_code'])['Admission_Deposit'].transform('min')

In [24]:
df['mean_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('mean')


In [25]:
df['sum_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('sum')
df['max_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('max')
df['min_Admission_Deposit_per_patient_hosp']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('min')

In [26]:
df['illness_count']=df.groupby('Hospital_code')['Severity of Illness'].transform('count')

In [27]:
df['bed_count']=df.groupby('Hospital_code')['Bed Grade'].transform('count')
df['room_count']=df.groupby('Hospital_code')['Available Extra Rooms in Hospital'].transform('count')
df['Department_count']=df.groupby('Hospital_code')['Department'].transform('count')
df['ward_count']=df.groupby('Hospital_code')['Ward_Type'].transform('count')
df['TOA_count']=df.groupby('Hospital_code')['Type of Admission'].transform('count')

In [28]:
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('mean')

In [29]:
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('sum')
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('max')
df['meanVisitors with Patient_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('min')

### Rank features

In [30]:
df["RANK"] = df.groupby("patientid")['patientid'].rank(method="first", ascending=True)
df["RANK_avg"] = df.groupby("patientid")['patientid'].rank(method="average", ascending=True)
df["RANK_max"] = df.groupby("patientid")['patientid'].rank(method="max", ascending=True)
df["RANK_min"] = df.groupby("patientid")['patientid'].rank(method="min", ascending=True)
df["RANK_DIFF"] = df['RANK_max'] - df['RANK_min']
df["RANK_Type_Admission"] = df.groupby(['patientid','Type of Admission'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_City_Code"] = df.groupby(['patientid','City_Code_Hospital'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_hosp_Code"] = df.groupby(['patientid','Hospital_code'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_hosp_type_Code"] = df.groupby(['patientid','Hospital_type_code'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_rooms"] = df.groupby(['patientid','Available Extra Rooms in Hospital'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Department"] = df.groupby(['patientid','Department'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Ward_Type"] = df.groupby(['patientid','Ward_Type'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Ward_Facility_Code"] = df.groupby(['patientid','Ward_Facility_Code'])['patientid'].rank(method='first',ascending=True)
df["RANK_Type_Hospital_region_code"] = df.groupby(['patientid','Hospital_region_code'])['patientid'].rank(method='first',ascending=True)
df['Rank_City_Code_Patient'] = df.groupby(['patientid','City_Code_Patient'])['patientid'].rank(method='first',ascending=True)
df['Rank_City_Code_Vist'] = df.groupby(['patientid','Visitors with Patient'])['patientid'].rank(method='first',ascending=True)
df['Rank_ill+admiss'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='first',ascending=True)
df['Rank_ill+admiss_max'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='max',ascending=True)
df['Rank_ill+admiss_min'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='min',ascending=True)
df['Rank_ill+admiss_avg'] = df.groupby(['patientid','Type of Admission','Severity of Illness'])['patientid'].rank(method='average',ascending=True)
df['Rank_ill+admiss_diff'] = df['Rank_ill+admiss_max'] - df['Rank_ill+admiss_min']
cols_rank = ['Type of Admission','City_Code_Hospital','Hospital_code','Hospital_type_code','Available Extra Rooms in Hospital','Department','Ward_Type','Ward_Facility_Code',
             'Hospital_region_code','City_Code_Patient','Visitors with Patient']
for i in cols_rank:
    df[f"RANK_{i}_min"] = df.groupby(['patientid',i])['patientid'].rank(method='min',ascending=True)
    df[f"RANK_{i}_max"] = df.groupby(['patientid',i])['patientid'].rank(method='max',ascending=True)
    df[f"RANK_{i}_avg"] = df.groupby(['patientid',i])['patientid'].rank(method='average',ascending=True)
    df[f'diff_{i}_max_min'] = df[f"RANK_{i}_max"] -  df[f"RANK_{i}_min"]

In [31]:
df['unique_hospital_visited']=df.groupby('patientid')['Hospital_code'].transform('nunique')
df['unique_hospitaltype_visited']=df.groupby('patientid')['Hospital_type_code'].transform('nunique')
df['count_visited_same_hospital']=df.groupby(['patientid','Hospital_code'])['Hospital_code'].transform('count')

df['mean_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('mean')
df['sum_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('sum')
df['max_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('max')
df['min_Admission_Deposit_per_patient']=df.groupby(['patientid'])['Admission_Deposit'].transform('min')

df['mean_visitors_per_patient']=df.groupby(['patientid'])['Visitors with Patient'].transform('mean')
df['unique city']=df.groupby('patientid')['City_Code_Hospital'].transform('nunique')
df['unique illness type']=df.groupby('patientid')['Severity of Illness'].transform('nunique')
df['unique admission type']=df.groupby('patientid')['Type of Admission'].transform('nunique')

df['mean_Admission_Deposit_per_patient_in_same_hospital']=df.groupby(['patientid','Hospital_code'])['Admission_Deposit'].transform('mean')

In [32]:
df['total_no_of_patients_in_hospital']=df.groupby('Hospital_code')['patientid'].transform('count')
df['total_no_of_patients_in_hospital_from_same_city']=df.groupby(['Hospital_code','City_Code_Patient'])['patientid'].transform('count')
df['total_no_of_patients_in_patientcity']=df.groupby('City_Code_Patient')['patientid'].transform('count')
df['total_no_of_patients_in_hospitalcity']=df.groupby('City_Code_Hospital')['patientid'].transform('count')

In [33]:
df['number_of_time_patient_visited']=df.groupby('patientid')['patientid'].transform('count')
df['Total_Hospitals_in_city']=df.groupby('City_Code_Hospital')['Hospital_code'].transform('nunique')
df['Total_Hospitals_in_city_of_same_type']=df.groupby(['City_Code_Hospital','Hospital_type_code'])['Hospital_code'].transform('nunique')

In [34]:
df['total_no_of_patients_per_Department']=df.groupby('Department')['patientid'].transform('count')
df['total_no_of_patients_per_Severity of Illness']=df.groupby('Severity of Illness')['patientid'].transform('count')
df['Total_Hospitals_in_region']=df.groupby('Hospital_region_code')['patientid'].transform('count')
df['total_no_of_patients_in_hospitalward']=df.groupby(['Hospital_code','Ward_Type'])['patientid'].transform('count')
df['total_no_of_patients_in_hospitaldepartment']=df.groupby(['Hospital_code','Department'])['patientid'].transform('count')
df['total_no_of_patients_in_hospitalBedGrade']=df.groupby(['Hospital_code','Bed Grade'])['patientid'].transform('count')
df['total_no_of_patients_in_hospitalSeverityofIllness']=df.groupby(['Hospital_code','Type of Admission'])['patientid'].transform('count')
df['unique_patientcity_in_hospital']=df.groupby('Hospital_code')['City_Code_Patient'].transform('nunique')
df['Average_deposit']=df.groupby('Hospital_code')['Available Extra Rooms in Hospital'].transform('mean')
df['Total_Hospitals_in_patientcity']=df.groupby('City_Code_Patient')['Hospital_code'].transform('nunique')
df['Total_Hospitals_in_region']=df.groupby('Hospital_region_code')['Hospital_code'].transform('nunique')

In [35]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)

In [36]:
train.head(2)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,mean_Admission_Deposit_per_illness,mean_Admission_Deposit_per_admission,mean_Admission_Deposit_per_bed,mean_Admission_Deposit_per_department,mean_Admission_Deposit_per_hosp,sum_Admission_Deposit_per_illness,sum_Admission_Deposit_per_admission,sum_Admission_Deposit_per_bed,sum_Admission_Deposit_per_department,sum_Admission_Deposit_per_hosp,max_Admission_Deposit_per_illness,max_Admission_Deposit_per_admission,max_Admission_Deposit_per_bed,max_Admission_Deposit_per_department,max_Admission_Deposit_per_hosp,min_Admission_Deposit_per_illness,min_Admission_Deposit_per_admission,min_Admission_Deposit_per_bed,min_Admission_Deposit_per_department,min_Admission_Deposit_per_hosp,mean_Admission_Deposit_per_patient_hosp,sum_Admission_Deposit_per_patient_hosp,max_Admission_Deposit_per_patient_hosp,min_Admission_Deposit_per_patient_hosp,illness_count,bed_count,room_count,Department_count,ward_count,TOA_count,meanVisitors with Patient_per_patient,RANK,...,diff_Department_max_min,RANK_Ward_Type_min,RANK_Ward_Type_max,RANK_Ward_Type_avg,diff_Ward_Type_max_min,RANK_Ward_Facility_Code_min,RANK_Ward_Facility_Code_max,RANK_Ward_Facility_Code_avg,diff_Ward_Facility_Code_max_min,RANK_Hospital_region_code_min,RANK_Hospital_region_code_max,RANK_Hospital_region_code_avg,diff_Hospital_region_code_max_min,RANK_City_Code_Patient_min,RANK_City_Code_Patient_max,RANK_City_Code_Patient_avg,diff_City_Code_Patient_max_min,RANK_Visitors with Patient_min,RANK_Visitors with Patient_max,RANK_Visitors with Patient_avg,diff_Visitors with Patient_max_min,unique_hospital_visited,unique_hospitaltype_visited,count_visited_same_hospital,mean_Admission_Deposit_per_patient,sum_Admission_Deposit_per_patient,max_Admission_Deposit_per_patient,min_Admission_Deposit_per_patient,mean_visitors_per_patient,unique city,unique illness type,unique admission type,mean_Admission_Deposit_per_patient_in_same_hospital,total_no_of_patients_in_hospital,total_no_of_patients_in_hospital_from_same_city,total_no_of_patients_in_patientcity,total_no_of_patients_in_hospitalcity,number_of_time_patient_visited,Total_Hospitals_in_city,Total_Hospitals_in_city_of_same_type,total_no_of_patients_per_Department,total_no_of_patients_per_Severity of Illness,Total_Hospitals_in_region,total_no_of_patients_in_hospitalward,total_no_of_patients_in_hospitaldepartment,total_no_of_patients_in_hospitalBedGrade,total_no_of_patients_in_hospitalSeverityofIllness,unique_patientcity_in_hospital,Average_deposit,Total_Hospitals_in_patientcity
0,1,8,2,3,2,3,2,0,0,2.0,31397,7.0,1,3.0,2,55,4911.0,0.0,4745.918406,4766.839769,4848.884671,4939.386738,4920.411809,385344845.0,8.025594e+08,855590549.0,202677856.0,25999456.0,11008.0,11920.0,11920.0,11920.0,9580.0,1800.0,1800.0,1800.0,1805.0,1958.0,4911.0,4911.0,4911.0,4911.0,2435,5284,5284,5284,5284,5284,2,1.0,...,9.0,1.0,5.0,3.0,4.0,1.0,5.0,3.0,4.0,1.0,4.0,2.5,3.0,1.0,14.0,7.5,13.0,1.0,14.0,7.5,13.0,10,7,1,5951.0,83314.0,8477.0,4449.0,2.0,7,1,3,4911.0,5284,340.0,33958.0,45544,14,6,6,41033,81195.0,10,2856,343,2285.0,1891,31,3.443036,32.0
1,2,2,2,5,2,2,2,2,0,2.0,31397,7.0,0,3.0,2,55,5954.0,4.0,4745.918406,4981.723373,4848.884671,4939.386738,4998.556685,385344845.0,1.084382e+09,855590549.0,202677856.0,36374497.0,11008.0,11008.0,11920.0,11920.0,10206.0,1800.0,1801.0,1800.0,1805.0,1848.0,5954.0,5954.0,5954.0,5954.0,3177,7277,7277,7277,7277,7277,2,2.0,...,9.0,1.0,8.0,4.5,7.0,1.0,5.0,3.0,4.0,1.0,4.0,2.5,3.0,1.0,14.0,7.5,13.0,1.0,14.0,7.5,13.0,10,7,1,5951.0,83314.0,8477.0,4449.0,2.0,7,1,3,5954.0,7277,493.0,33958.0,44395,14,4,2,41033,81195.0,10,6206,658,2808.0,3701,33,2.682149,32.0


In [37]:
# groups_by_patient_id_list = train['patientid'].copy().tolist()

In [38]:
x=train.drop(columns={'case_id','Stay'},axis=1)
y=train.loc[:,['Stay']]
test=test.drop(columns={'case_id','Stay'},axis=1)

### GroupKFold

In [39]:
# %%time
# err = [] 
# y_pred_tot_lgm = []


# fold = GroupKFold(n_splits=5)
# i = 1

# for train_index, test_index in fold.split(x, y, groups = groups_by_patient_id_list):
#     x_train, x_val = x.iloc[train_index], x.iloc[test_index]
#     y_train, y_val = y.iloc[train_index], y.iloc[test_index]
#     m = LGBMClassifier(n_estimators=10000,
#                        random_state=2020,
#                        learning_rate=0.1,
#                        num_leaves=125,
#                        colsample_bytree=0.5
#                        )
#     m.fit(x_train, y_train,eval_set=[(x_val, y_val)],eval_metric='multi_error', early_stopping_rounds=30,verbose=100)
#     pred_y = m.predict(x_val)
#     print(i, " err_lgm: ", accuracy_score(y_val,pred_y))
#     err.append(accuracy_score(y_val,pred_y))
#     pred_test = m.predict(test)
#     i = i + 1
#     y_pred_tot_lgm.append(pred_test)
# sum(err)/5    

In [40]:
#43.39

### StratifiedKFOLD

In [41]:
# %%time
# err = [] 
# y_pred_tot_lgm = []


# fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
# i = 1

# for train_index, test_index in fold.split(x, y):
#     x_train, x_val = x.iloc[train_index], x.iloc[test_index]
#     y_train, y_val = y.iloc[train_index], y.iloc[test_index]
#     m = CatBoostClassifier(n_estimators=10000,
#                        random_state=2020,
#                        eval_metric='Accuracy',
#                        learning_rate=0.1,
#                        depth=8,    
#                        task_type='GPU'
#                        #num_leaves=64
                       
#                        )
#     m.fit(x_train, y_train,eval_set=[(x_val, y_val)], early_stopping_rounds=100,verbose=100)
#     pred_y = m.predict(x_val)
#     print(i, " err_lgm: ", accuracy_score(y_val,pred_y))
#     err.append(accuracy_score(y_val,pred_y))
#     pred_test = m.predict(test)
#     i = i + 1
#     y_pred_tot_lgm.append(pred_test)
# sum(err)/5    

In [42]:
#0.4388

In [58]:
%%time
err = [] 
y_pred_tot_lgm = np.zeros((len(test), 11))


fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)
i = 1

for train_index, test_index in fold.split(x, y):
    x_train, x_val = x.iloc[train_index], x.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    m = CatBoostClassifier(n_estimators=10000,
                       random_state=2020,
                       eval_metric='Accuracy',
                       learning_rate=0.08,
                       depth=8,
                       bagging_temperature=0.3,
                       task_type='GPU'
                       #num_leaves=64
                       
                       )
    m.fit(x_train, y_train,eval_set=[(x_val, y_val)], early_stopping_rounds=100,verbose=200)
    pred_y = m.predict(x_val)
    print(i, " err_lgm: ", accuracy_score(y_val,pred_y))
    err.append(accuracy_score(y_val,pred_y))
    y_pred_tot_lgm+= m.predict_proba(test)
    i = i + 1
y_pred_tot_lgm=y_pred_tot_lgm/10
sum(err)/10

0:	learn: 0.3810206	test: 0.3811550	best: 0.3811550 (0)	total: 81.1ms	remaining: 13m 31s
200:	learn: 0.4557448	test: 0.4354981	best: 0.4354981 (200)	total: 6.15s	remaining: 4m 59s
400:	learn: 0.4803376	test: 0.4389524	best: 0.4395962 (345)	total: 12.4s	remaining: 4m 56s
600:	learn: 0.5038351	test: 0.4406953	best: 0.4409622 (534)	total: 18.3s	remaining: 4m 45s
bestTest = 0.4411192061
bestIteration = 631
Shrink model to first 632 iterations.
1  err_lgm:  0.4411192061298832
0:	learn: 0.3817036	test: 0.3783915	best: 0.3783915 (0)	total: 33.5ms	remaining: 5m 35s
200:	learn: 0.4563651	test: 0.4344618	best: 0.4344775 (197)	total: 5.98s	remaining: 4m 51s
400:	learn: 0.4802120	test: 0.4394705	best: 0.4394705 (400)	total: 13.2s	remaining: 5m 15s
600:	learn: 0.5023984	test: 0.4396590	best: 0.4402713 (525)	total: 19.7s	remaining: 5m 7s
bestTest = 0.4402713227
bestIteration = 525
Shrink model to first 526 iterations.
2  err_lgm:  0.4402713226981535
0:	learn: 0.3811187	test: 0.3810608	best: 0.381060

0.44113139967259213

In [ ]:
44.1941

In [50]:
stay_map = {2:'21-30',
 1:'11-20',
 3:'31-40',
 5:'51-60',
 0:'0-10',
 4:'41-50',
 7:'71-80',
 10:'More than 100 Days',
 8:'81-90',
 9:'91-100',
 6:'61-70'}

#### Predicting from average

In [51]:
sub['Stay'] = [np.argmax(x) for x in y_pred_tot_lgm]
sub['Stay']=sub['Stay'].map(stay_map)
sub['Stay'].value_counts(normalize=True)
sub.to_csv('lgbm_final_data.csv',index=False)

### Predicting using mode

In [100]:
# %%time
# d = pd.DataFrame()
# for i in range(5):
#     d = pd.concat([d,pd.DataFrame(y_pred_tot_lgm[i])],axis=1)
# d.columns=['1','2','3','4','5']

# re = d.mode(axis=1)[0]
# sub['Stay']=re
# sub['Stay']=sub['Stay'].astype(int)
# sub['Stay']=sub['Stay'].map(stay_map)
# sub.to_csv('lgb_baseline.csv',index=False)

In [101]:
# %%time
# d = pd.DataFrame()
# for i in range(10):
#     d = pd.concat([d,pd.DataFrame(y_pred_tot_lgm[i])],axis=1)
# d.columns=['1','2','3','4','5','6','7','8','9','10'
#           ]

# re = d.mode(axis=1)[0]

# sub['Stay']=re
# sub['Stay']=sub['Stay'].astype(int)
# sub['Stay']=le.inverse_transform(sub['Stay'])
# sub.to_csv('lgb_baseline.csv',index=False)